In [ ]:
!pip install pathway --quiet
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pathway as pw
from pathway.io import csv

file_path = "/content/drive/MyDrive/Data_sets/dataset.csv"

# Load the CSV with inferred schema
data = csv.read(
    file_path,
    schema=pw.schema_from_csv(file_path),
    mode="static",
)

# Data validation (based on your actual data structure)
data = data.filter(
    (data.Occupancy >= 0) &
    (data.QueueLength >= 0) &
    (data.Capacity > 0) &
    ((data.IsSpecialDay == 0) | (data.IsSpecialDay == 1))
)

# Step 2: Extract hour and create datetime components

def extract_hour_from_time(time_str: str) -> int:
    """Extract hour from time string format HH:MM:SS"""
    try:
        return int(time_str.split(':')[0])
    except:
        return 12  # Default to noon if parsing fails

def create_datetime_string(date_str: str, time_str: str) -> str:
    """Combine date and time strings"""
    return date_str + " " + time_str

data_with_time = data.with_columns(
    datetime_str=pw.apply(create_datetime_string, data.LastUpdatedDate, data.LastUpdatedTime),
    hour=pw.apply(extract_hour_from_time, data.LastUpdatedTime),
    day=data.LastUpdatedDate,  # Use date string directly for now
)

# Step 3: Add vehicle type weights and traffic weights

def get_vehicle_weight(vehicle_type: str) -> float:
    """Assign weights based on vehicle type"""
    if vehicle_type == "truck":
        return 1.5  # Trucks take more space/create more demand
    elif vehicle_type == "car":
        return 1.0
    elif vehicle_type == "bike":
        return 0.5  # Bikes take less space
    else:
        return 1.0  # Default for unknown types

def get_traffic_weight(traffic_condition: str) -> float:
    """Assign weights based on traffic condition"""
    if traffic_condition == "high":
        return 1.3  # High traffic increases demand for parking
    elif traffic_condition == "medium":
        return 1.1
    elif traffic_condition == "low":
        return 1.0
    else:
        return 1.0  # Default

# Add weights to the data
weighted_data = data_with_time.with_columns(
    vehicle_weight=pw.apply(get_vehicle_weight, data_with_time.VehicleType),
    traffic_weight=pw.apply(get_traffic_weight, data_with_time.TrafficConditionNearby),
    occupancy_rate=data_with_time.Occupancy / data_with_time.Capacity
)

# Step 4: Compute demand score (Model 2 implementation)

def calculate_demand(occupancy_rate: float, queue: int, traffic_weight: float,
                    special_day: int, vehicle_weight: float, hour: int) -> float:
    """
    Advanced demand function as per Model 2 requirements
    Demand = α·(Occupancy/Capacity) + β·QueueLength + γ·Traffic + δ·IsSpecialDay + ε·VehicleTypeWeight + ζ·HourFactor
    """
    # Coefficients (you can tune these based on business logic)
    alpha = 0.4    # Occupancy rate weight
    beta = 0.25    # Queue length weight
    gamma = 0.15   # Traffic condition weight
    delta = 0.1    # Special day weight
    epsilon = 0.05 # Vehicle type weight
    zeta = 0.05    # Hour factor weight

    # Peak hours (9-11 AM, 1-3 PM, 5-7 PM) get higher weight
    hour_factor = 1.2 if hour in [9, 10, 11, 13, 14, 15, 17, 18, 19] else 1.0

    demand = (alpha * occupancy_rate +
              beta * queue +
              gamma * traffic_weight +
              delta * special_day +
              epsilon * vehicle_weight +
              zeta * hour_factor)

    return demand

scored = weighted_data.with_columns(
    demand_score=pw.apply(
        calculate_demand,
        weighted_data.occupancy_rate,
        weighted_data.QueueLength,
        weighted_data.traffic_weight,
        weighted_data.IsSpecialDay,
        weighted_data.vehicle_weight,
        weighted_data.hour,
    )
)

# Step 5: Simple normalization without groupby (fallback approach)

step2 = scored.with_columns(
    normalized_demand=scored.demand_score / 2.0
)

# Step 6: Price calculation (Model 2 implementation)

BASE_PRICE = 10
LAMBDA = 0.8

# Ensure price variations are bounded (0.5x to 2x base price as per requirements)
final = step2.with_columns(
    price_multiplier=1 + (LAMBDA * step2.normalized_demand),
    unbounded_price=BASE_PRICE * (1 + LAMBDA * step2.normalized_demand)
).with_columns(
    # Bound the price between 0.5x and 2x base price
    price=pw.if_else(
        pw.this.unbounded_price > 2 * BASE_PRICE,
        2 * BASE_PRICE,
        pw.if_else(
            pw.this.unbounded_price < 0.5 * BASE_PRICE,
            0.5 * BASE_PRICE,
            pw.this.unbounded_price
        )
    )
)

# Step 7: Output comprehensive results

output_data = final.select(
    final.ID,
    final.SystemCodeNumber,
    final.datetime_str,
    final.hour,
    final.day,
    final.Latitude,
    final.Longitude,
    final.Capacity,
    final.Occupancy,
    final.occupancy_rate,
    final.QueueLength,
    final.VehicleType,
    final.TrafficConditionNearby,
    final.IsSpecialDay,
    final.vehicle_weight,
    final.traffic_weight,
    final.demand_score,
    final.normalized_demand,
    final.price_multiplier,
    final.price,
)

# Write to CSV (this defines what Pathway will write at pipeline execution)
csv.write(output_data, "model2_output.csv")

# Run the Pathway pipeline (this step actually processes and writes the CSV)
pw.run()

print("Model 2 processing complete!")
print(f"Base price: ${BASE_PRICE}")
print(f"Price range: ${0.5 * BASE_PRICE:.2f} - ${2 * BASE_PRICE:.2f}")
print("Output saved to model2_output.csv")


In [ ]:
# Install necessary packages
!pip install bokeh panel --quiet

import pandas as pd
import bokeh.plotting
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category20
import panel as pn

# Activate Panel (for interactive plotting)
pn.extension()

# Load the Pathway Model 2 output
df = pd.read_csv("model2_output.csv")

# Convert datetime string to datetime type for plotting
df['timestamp'] = pd.to_datetime(df['datetime_str'], errors='coerce')

# Drop rows with missing timestamps or prices (to avoid plotting issues)
df = df.dropna(subset=['timestamp', 'price', 'SystemCodeNumber'])

# Create Bokeh figure
fig = bokeh.plotting.figure(
    height=500,
    width=900,
    title="Parking Lot-wise Dynamic Pricing (Model 2 Output)",
    x_axis_type="datetime",
    tools="pan,wheel_zoom,box_zoom,reset,save"
)

# Assign colors to parking lots (up to 20 lots)
colors = Category20[20]
lot_ids = sorted(df['SystemCodeNumber'].unique())  # Unique lots

# lot each lot separately
for idx, lot_id in enumerate(lot_ids):
    lot_data = df[df['SystemCodeNumber'] == lot_id].sort_values('timestamp')
    source = ColumnDataSource(lot_data)

    fig.line(
        "timestamp", "price",
        source=source,
        line_width=2,
        color=colors[idx % len(colors)],
        legend_label=f"Lot {lot_id}"
    )

    fig.circle(
        "timestamp", "price",
        source=source,
        size=5,
        color=colors[idx % len(colors)]
    )

# Final plot settings
fig.xaxis.axis_label = "Timestamp"
fig.yaxis.axis_label = "Predicted Price"
fig.legend.location = "top_left"
fig.legend.click_policy = "hide"  # Allow interactive hide/show per lot

# Display the interactive plot
pn.Column(fig).servable()
